### import modul dan library

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import pandas as pd
import tensorflow_model_analysis as tfma
from sklearn.preprocessing import LabelEncoder
from tfx.proto import trainer_pb2
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.components import Evaluator
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 

Load encoder dan cek dataset

In [2]:
label_encoder = LabelEncoder()

df = pd.read_csv("raw_data/review_dana_labelled.csv")
df

,userName,score,at,content,sentimen
0,Elisya Kasni,5,2024-02-15 11:24:56,Bagus,POSITIVE
1,Rusman Man,2,2024-02-15 11:24:03,Dana mmg keren mantap.,POSITIVE
2,Qiliw Sadega,1,2024-02-15 11:23:34,Saya ngajuin upgrade dana premium krna ktp say...,NEGATIVE
3,Kijutjrv2 Kijut,3,2024-02-15 11:22:46,Kocak mana diskon nya ml malah eror segala kag...,NEGATIVE
4,Fifi Alfiyah,1,2024-02-15 11:21:34,Saldo hilang karena no lama Hilang ganti no sa...,NEGATIVE
...,...,...,...,...,...
49995,Marha Marha,5,2023-12-31 16:34:55,Bagus,POSITIVE
49996,Unique Laudy,1,2023-12-31 16:33:47,"error, tidak bisa masuk",NEGATIVE
49997,supendi supendi,5,2023-12-31 16:30:08,Cukup membantu,POSITIVE
49998,hasansend farm,1,2023-12-31 16:28:51,"tolong pihak dana knp saldo saya hilang,ada yg...",NEUTRAL


Mengambil fitur yang digunakan saja, serta melakukan encode untuk label

In [3]:
df = df[["content", "sentimen"]].copy()
df["sentimen"] = label_encoder.fit_transform(df["sentimen"])
df.head()

,content,sentimen
0,Bagus,2
1,Dana mmg keren mantap.,2
2,Saya ngajuin upgrade dana premium krna ktp say...,0
3,Kocak mana diskon nya ml malah eror segala kag...,0
4,Saldo hilang karena no lama Hilang ganti no sa...,0


Menyimpan data kembali ke CSV

In [4]:
df.to_csv("data/data_encoded.csv", index=False)

### Mengatur dan mempersiapkan variabel untuk pipeline

In [5]:
PIPELINE_NAME = "sentimen-pipeline"
SCHEMA_PIPELINE_NAME = "sentimen-tfdv-schema"

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [6]:
DATA_ROOT = "data"

In [7]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

### Tahapan Data Ingestion

Mengimpor dan membagi data untuk pipeline

In [8]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [9]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 10
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

Menghasilkan dan menampilkan statistik deskriptif dari data

In [10]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 11
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [11]:
interactive_context.show(statistics_gen.outputs["statistics"])

Membuat skema berdasarkan statistik

In [12]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 12
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [13]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'content',BYTES,required,,-
'sentimen',INT,required,,-


Mengecek anomali di dalam data

In [14]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 13
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [15]:
interactive_context.show(example_validator.outputs['anomalies'])

### Tahapan Data Preprocessing

inisialisasi nama file py

In [16]:
TRANSFORM_MODULE_FILE = "sentimen_transform.py"

Membuat dan menyimpan fungsi preprocessing dalam Python

In [17]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf

LABEL_KEY = "sentimen"
FEATURE_KEY = "content"
 
def transformed_name(key):
    return key + "_xf"

def preprocessing_fn(inputs):
    outputs = {}

    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])

    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    return outputs


Overwriting sentimen_transform.py


Melakukan proses preprocessing terhadap data

In [18]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\sentimen-pipeline\Transform\transform_graph\14\.temp_path\tftransform_tmp\f67a152a2f1c46898bd48e2097ecbb4c\assets


INFO:tensorflow:Assets written to: pipelines\sentimen-pipeline\Transform\transform_graph\14\.temp_path\tftransform_tmp\f67a152a2f1c46898bd48e2097ecbb4c\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 14
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

### Tahap Pengembangan Model

Inisialisasi nama file py

In [19]:
TRAINER_MODULE_FILE = "sentimen_trainer.py"

Membuat dan menyimpan fungsi untuk mengatur bentuk input, ventorizer teks, struktur model, dan running model.

In [20]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "sentimen"
FEATURE_KEY = "content"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    print("Fitur yang tersedia:", transform_feature_spec.keys())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    

    return dataset
 
# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")
 
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(3, activation='softmax', name="probabilities")(x)
    
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=['accuracy']
    
    )
    
    # print(model)
    model.summary()
    return model 
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 100, 
            validation_steps= 100,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting sentimen_trainer.py


Melakukan proses permodelan

In [21]:
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Fitur yang tersedia: dict_keys(['content_xf', 'sentimen_xf'])
Fitur yang tersedia: dict_keys(['content_xf', 'sentimen_xf'])
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 content_xf (InputLayer)     [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
   

INFO:tensorflow:Assets written to: pipelines\sentimen-pipeline\Trainer\model\15\Format-Serving\assets


100/100 [==============================] - 5s 36ms/step - loss: 0.7442 - accuracy: 0.6728 - val_loss: 0.5677 - val_accuracy: 0.7600
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 0.5341 - accuracy: 0.7730
Epoch 2: val_accuracy improved from 0.76000 to 0.76344, saving model to pipelines\sentimen-pipeline\Trainer\model\15\Format-Serving
INFO:tensorflow:Assets written to: pipelines\sentimen-pipeline\Trainer\model\15\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\sentimen-pipeline\Trainer\model\15\Format-Serving\assets


100/100 [==============================] - 3s 30ms/step - loss: 0.5341 - accuracy: 0.7730 - val_loss: 0.5551 - val_accuracy: 0.7634
Epoch 3/10
 96/100 [===========================>..] - ETA: 0s - loss: 0.4916 - accuracy: 0.7834
Epoch 3: val_accuracy improved from 0.76344 to 0.79172, saving model to pipelines\sentimen-pipeline\Trainer\model\15\Format-Serving
INFO:tensorflow:Assets written to: pipelines\sentimen-pipeline\Trainer\model\15\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\sentimen-pipeline\Trainer\model\15\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\sentimen-pipeline\Trainer\model\15\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 15
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Tahapan Analisis dan Evaluasi

Mengambil model terbaru

In [22]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 16
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

Mengevaluasi performa model dengan metrik ExampleCount, SparseCategoricalAccuracy, MeanPrediction, MeanLabel, MAE, RMSE, ConfusionMatrixPlot, dan CalibrationPlot.

In [23]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            label_key='sentimen'
        )
    ],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                
                tfma.MetricConfig(
                    class_name='SparseCategoricalAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.6}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': 0.0001}
                        )
                    )
                ),

                tfma.MetricConfig(class_name='MeanPrediction'),
                tfma.MetricConfig(class_name='MeanLabel'),

                tfma.MetricConfig(class_name='MeanAbsoluteError'),
                tfma.MetricConfig(class_name='RootMeanSquaredError'),

                tfma.MetricConfig(class_name='ConfusionMatrixPlot'),

                tfma.MetricConfig(class_name='CalibrationPlot')
            ]
        )
    ]
)

Menjalankan proses evaluasi model

In [24]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator, enable_cache=False)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 17
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

Menampilkan hasil evaluasi model dengan metrik yang telah ditentukan

In [25]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

### Tahap Pusher Model

In [26]:
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/sentimen-model'))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 18
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

### run docker

docker build -t sentimen-tf-serving .

docker run -p 8080:8501 sentimen-tf-serving